In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip3 install git+https://github.com/paoloripamonti/word2vec-keras
!pip install mlflow
!pip3 install --user -U nltk
!pip3 install --user -U numpy
from word2vec_keras import Word2VecKeras
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np
import nltk
import string
import re
import ast # abstract syntax tree: https://docs.python.org/3/library/ast.html
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
import re as re
import nltk

%matplotlib inline


  Cloning https://github.com/paoloripamonti/word2vec-keras to /tmp/pip-req-build-n337uq8w
  Running command git clone -q https://github.com/paoloripamonti/word2vec-keras /tmp/pip-req-build-n337uq8w
  Created wheel for word2vec-keras: filename=word2vec_keras-0.1-cp36-none-any.whl size=6441 sha256=8a9b04dd6a62ede745433091548a2c8fb014a3bd0106221f8d1171d11a7d3fcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-0plzlqow/wheels/8d/5c/17/4b39b98e4ed57d4a8d3e84e97f45841f70096ca28a5eabeb31
Successfully built word2vec-keras
     |████████████████████████████████| 14.6MB 1.6MB/s 
     |████████████████████████████████| 1.1MB 62.7MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 460kB 61.1MB/s 
     |████████████████████████████████| 92kB 13.2MB/s 
     |████████████████████████████████| 143kB 62.4MB/s 
     |████████████████████████████████| 471kB 60.3MB/s 
     |████████████████████████████████| 71kB 12.7MB/s 
     |████████████████████████████

Using TensorFlow backend.


In [14]:
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
column_names = ['label','body_text']
raw_data2 = pd.read_csv('/content/drive/My Drive/sentiment/sentiment.csv', encoding = "latin-1")
raw_data2.columns = column_names
raw_data2 = raw_data2.sample(frac=1.0)
raw_data2.head()


,label,body_text
7993,positive,muthis senaryo muthis bir son kesinlikle izlen...
910,negative,ben beenmedim arkada ok ktyd skntdan patladm
3919,negative,film alabilcek kadar basarili bir film deil be...
7226,positive,kesinlikle otesi yok daha iyisi olamazrun forr...
8059,positive,ne demeli nereden baslanmali her izleyisimde b...


In [0]:
class Preprocessing(object):
    def __init__(self, data, target_column_name='body_text_clean'):
        self.data = data
        self.feature_name = target_column_name
        
    def remove_punctuation(self, text):
        text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
        return text_nopunct
    
    def tokenize(self, text):
        # Match one or more characters which are not word character
        tokens = re.split((r'(\W+)'), text) 
        return tokens
    
    def remove_stopwords(self, tokenized_list):
        # Remove all English Stopwords
        stopword = nltk.corpus.stopwords.words('turkish')
        text = [word for word in tokenized_list if word not in stopword]
        return text
    def stemming(self, tokenized_text):
        ps = nltk.PorterStemmer()
        text = [ps.stem(word) for word in tokenized_text]
        return text
    
    def lemmatizing(self, tokenized_text):
        wn = nltk.WordNetLemmatizer()
        text = [wn.lemmatize(word) for word in tokenized_text]
        return text
    
    def tokens_to_string(self, tokens_string):
        try:
            list_obj = ast.literal_eval(tokens_string)
            text = " ".join(list_obj)
        except:
            text = None
        return text
    
    def dropna(self):
        feature_name = self.feature_name
        if self.data[feature_name].isnull().sum() > 0:
            column_list=[feature_name]
            self.data = self.data.dropna(subset=column_list)
            return self.data
        
    def preprocessing(self):
        self.data['body_text_nopunc'] = self.data['body_text'].apply(lambda x: self.remove_punctuation(x))
        self.data['body_text_tokenized'] = self.data['body_text_nopunc'].apply(lambda x: self.tokenize(x.lower())) 
        self.data['body_text_nostop'] = self.data['body_text_tokenized'].apply(lambda x: self.remove_stopwords(x))
        self.data['body_text_stemmed'] = self.data['body_text_nostop'].apply(lambda x: self.stemming(x))
        self.data['body_text_lemmatized'] = self.data['body_text_nostop'].apply(lambda x: self.lemmatizing(x))
        
        # save cleaned dataset into csv file and load back
        self.save()
        self.load()
        
        self.data[self.feature_name] = self.data['body_text_lemmatized'].apply(lambda x: self.tokens_to_string(x))
        
        self.dropna()
        
        drop_columns = ['body_text_nopunc', 'body_text_tokenized', 'body_text_nostop', 'body_text_stemmed', 'body_text_lemmatized'] 
        self.data.drop(drop_columns, axis=1, inplace=True)
        return self.data
    
    def save(self, filepath="/content/drive/My Drive/spam/sentiment_cleaned.csv"):
        self.data.to_csv(filepath, index=False, sep=',')  
        
    def load(self, filepath="/content/drive/My Drive/spam/sentiment_cleaned.csv"):
        self.data = pd.read_csv(filepath)
        return self.data

In [0]:
class SentimentClassifier(object):
    def __init__(self):
        self.model = Word2VecKeras()
        
    def load_data(self):
        column_names = ['label', 'body_text']
        data = pd.read_csv('/content/drive/My Drive/sentiment/sentiment.csv', encoding = "ISO-8859-1")
        data.columns = column_names
        self.raw_data = data.sample(frac=1.0) 
        
        return self.raw_data
    
    def split_data(self):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size=0.25, random_state=42)
        
    def numpy_to_list(self):
        self.x_train = self.x_train.tolist()
        self.y_train = self.y_train.tolist()
        self.x_test  = self.x_test.tolist()
        self.y_test  = self.y_test.tolist()
    
    def prepare_data(self, feature, label='label'):
        self.load_data()
        pp = Preprocessing(self.raw_data)
        self.data = pp.preprocessing()
        self.x = self.data[feature].values
        self.y = self.data[label].values
        self.split_data()
        self.numpy_to_list()
        
        return self.data
        
    def train_model(self):
        self.w2v_size = 300
        self.w2v_min_count = 1 # 5
        self.w2v_epochs = 100
        self.k_epochs = 10 # 32
        self.k_lstm_neurons = 512
        self.k_max_sequence_len = 1000
        
        self.model.train(self.x_train, self.y_train, 
            w2v_size=self.w2v_size, 
            w2v_min_count=self.w2v_min_count, 
            w2v_epochs=self.w2v_epochs, 
            k_epochs=self.k_epochs, 
            k_lstm_neurons=self.k_lstm_neurons, 
            k_max_sequence_len=self.k_max_sequence_len, 
            k_hidden_layer_neurons=[])
        
    def evaluate(self):
        self.result = self.model.evaluate(self.x_test, self.y_test)
        self.accuracy = self.result["ACCURACY"]
        self.clf_report_df = pd.DataFrame(self.result["CLASSIFICATION_REPORT"])
        self.cnf_matrix = self.result["CONFUSION_MATRIX"]
        return self.result
    
    def predict(self, idx=1):
        print("LABEL:", self.y_test[idx])
        print("TEXT :", self.x_test[idx])
        print("/n============================================")
        print("PREDICTION:", self.model.predict(self.x_test[idx]))
        
    def mlFlow(self, feature='body_text_clean'):
        np.random.seed(40)  
        with mlflow.start_run():
            self.prepare_data(feature=feature) # feature should be 'body_text' if no need to preprocessing
            self.train_model()
            self.evaluate()
            self.predict()
            mlflow.log_param("feature", feature) 
            mlflow.log_param("w2v_size", self.w2v_size)  
            mlflow.log_param("w2v_min_count", self.w2v_min_count)
            mlflow.log_param("w2v_epochs", self.w2v_epochs)
            mlflow.log_param("k_lstm_neurons", self.k_lstm_neurons)
            mlflow.log_param("k_max_sequence_len", self.k_max_sequence_len)
            mlflow.log_metric("accuracy", self.accuracy)
            mlflow.sklearn.log_model(self.model, "Word2Vec-Keras")

In [9]:
spam_clf = SentimentClassifier() # epoch 5 preprocessed data
spam_clf.mlFlow(feature='body_text_clean')



2020-01-20 12:49:23,024 : INFO : Build & train Word2Vec model
2020-01-20 12:49:23,025 : INFO : collecting all words and their counts
2020-01-20 12:49:23,026 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-20 12:49:23,061 : INFO : collected 23660 word types from a corpus of 141543 raw words and 7991 sentences
2020-01-20 12:49:23,061 : INFO : Loading a fresh vocabulary
2020-01-20 12:49:23,100 : INFO : effective_min_count=1 retains 23660 unique words (100% of original 23660, drops 0)
2020-01-20 12:49:23,101 : INFO : effective_min_count=1 leaves 141543 word corpus (100% of original 141543, drops 0)
2020-01-20 12:49:23,174 : INFO : deleting the raw counts dictionary of 23660 items
2020-01-20 12:49:23,175 : INFO : sample=0.001 downsamples 32 most-common words
2020-01-20 12:49:23,175 : INFO : downsampling leaves estimated 121315 word corpus (85.7% of prior 141543)
2020-01-20 12:49:23,216 : INFO : estimated required memory for 23660 words and 300 dimensions: 

2020-01-20 12:50:01,438 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-01-20 12:50:01,450 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-01-20 12:50:01,457 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2020-01-20 12:50:01,468 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



...

2020-01-20 12:50:10,648 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



2020-01-20 12:50:10,724 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/5
7991/7991 [==============================] - 96s 12ms/step - loss: 0.4183 - acc: 0.8154
Epoch 2/5
7991/7991 [==============================] - 94s 12ms/step - loss: 0.3060 - acc: 0.8736
Epoch 3/5
7991/7991 [==============================] - 94s 12ms/step - loss: 0.2748 - acc: 0.8846
Epoch 4/5
7991/7991 [==============================] - 94s 12ms/step - loss: 0.2562 - acc: 0.8955
Epoch 5/5
7991/7991 [==============================] - 94s 12ms/step - loss: 0.2310 - acc: 0.9030


2020-01-20 12:58:01,637 : INFO : Done


2664/2664 [==============================] - 52s 20ms/step
LABEL: positive
TEXT : gorselligi     ust   seviyelere   ulasmis   ufak   kizin   muhtesem   oyunculuguyla   suslenmis   muazzam   bir   sinema   soleni
/n============================================
PREDICTION: {'label': 'positive', 'confidence': 0.4621184170246124, 'elapsed_time': 0.6001675128936768}


In [10]:
spam_clf.predict(2)

LABEL: negative
TEXT : sadece   kafa   dagitmak   icin   izlenilir   bir   orjinallik   yok
/n============================================
PREDICTION: {'label': 'negative', 'confidence': 0.8223912715911865, 'elapsed_time': 0.6863346099853516}


In [11]:
spam_clf = SentimentClassifier() # epoch 5 not processed data
spam_clf.mlFlow(feature='body_text')

2020-01-20 12:59:03,501 : INFO : Build & train Word2Vec model
2020-01-20 12:59:03,504 : INFO : collecting all words and their counts
2020-01-20 12:59:03,505 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-20 12:59:03,540 : INFO : collected 23667 word types from a corpus of 157613 raw words and 7991 sentences
2020-01-20 12:59:03,541 : INFO : Loading a fresh vocabulary
2020-01-20 12:59:03,778 : INFO : effective_min_count=1 retains 23667 unique words (100% of original 23667, drops 0)
2020-01-20 12:59:03,779 : INFO : effective_min_count=1 leaves 157613 word corpus (100% of original 157613, drops 0)
2020-01-20 12:59:03,848 : INFO : deleting the raw counts dictionary of 23667 items
2020-01-20 12:59:03,849 : INFO : sample=0.001 downsamples 38 most-common words
2020-01-20 12:59:03,850 : INFO : downsampling leaves estimated 131462 word corpus (83.4% of prior 157613)
2020-01-20 12:59:03,898 : INFO : estimated required memory for 23667 words and 300 dimensions: 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 300)         7100400   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1539      
Total params: 8,766,963
Trainable params: 1,666,563
Non-trainable params: 7,100,400
_________________________________________________________________
Epoch 1/5
7991/7991 [==============================] - 96s 12ms/step - loss: 0.4304 - acc: 0.8103
Epoch 2/5
7991/7991 [==============================] - 96s 12ms/step - loss: 0.3115 - acc: 0.8665
Epoch 3/5
7991/7991 [==============================] - 96s 12ms/step - loss: 0.2735 - acc: 0.8864
Epoch 4/5
7991/7991 [==============================] - 96s 12ms/step - l

2020-01-20 13:07:45,962 : INFO : Done


2664/2664 [==============================] - 51s 19ms/step
LABEL: positive
TEXT : gorselligi en ust seviyelere ulasmis ufak kizin muhtesem oyunculuguyla suslenmis muazzam bir sinema soleni
/n============================================
PREDICTION: {'label': 'positive', 'confidence': 0.46075788140296936, 'elapsed_time': 0.6075527667999268}


In [16]:
spam_clf = SentimentClassifier()
spam_clf.mlFlow(feature='body_text_clean') # epoch 10  preprocessed data

2020-01-20 13:10:38,129 : INFO : Build & train Word2Vec model
2020-01-20 13:10:38,130 : INFO : collecting all words and their counts
2020-01-20 13:10:38,131 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-20 13:10:38,165 : INFO : collected 23660 word types from a corpus of 141543 raw words and 7991 sentences
2020-01-20 13:10:38,166 : INFO : Loading a fresh vocabulary
2020-01-20 13:10:38,201 : INFO : effective_min_count=1 retains 23660 unique words (100% of original 23660, drops 0)
2020-01-20 13:10:38,201 : INFO : effective_min_count=1 leaves 141543 word corpus (100% of original 141543, drops 0)
2020-01-20 13:10:38,272 : INFO : deleting the raw counts dictionary of 23660 items
2020-01-20 13:10:38,273 : INFO : sample=0.001 downsamples 32 most-common words
2020-01-20 13:10:38,274 : INFO : downsampling leaves estimated 121315 word corpus (85.7% of prior 141543)
2020-01-20 13:10:38,314 : INFO : estimated required memory for 23660 words and 300 dimensions: 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 300)         7098300   
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 1539      
Total params: 8,764,863
Trainable params: 1,666,563
Non-trainable params: 7,098,300
_________________________________________________________________
Epoch 1/10
7991/7991 [==============================] - 98s 12ms/step - loss: 0.4143 - acc: 0.8173
Epoch 2/10
7991/7991 [==============================] - 98s 12ms/step - loss: 0.3053 - acc: 0.8736
Epoch 3/10
7991/7991 [==============================] - 98s 12ms/step - loss: 0.2758 - acc: 0.8839
Epoch 4/10
7991/7991 [==============================] - 101s 13ms/ste

2020-01-20 13:28:12,844 : INFO : Done


2664/2664 [==============================] - 52s 19ms/step
LABEL: positive
TEXT : gorselligi     ust   seviyelere   ulasmis   ufak   kizin   muhtesem   oyunculuguyla   suslenmis   muazzam   bir   sinema   soleni
/n============================================
PREDICTION: {'label': 'negative', 'confidence': 0.6502218246459961, 'elapsed_time': 0.5991842746734619}


In [17]:
spam_clf = SentimentClassifier() # epoch 10 not preprocessed data
spam_clf.mlFlow(feature='body_text')

2020-01-20 13:29:14,846 : INFO : Build & train Word2Vec model
2020-01-20 13:29:14,850 : INFO : collecting all words and their counts
2020-01-20 13:29:14,851 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-20 13:29:14,886 : INFO : collected 23667 word types from a corpus of 157613 raw words and 7991 sentences
2020-01-20 13:29:14,886 : INFO : Loading a fresh vocabulary
2020-01-20 13:29:14,930 : INFO : effective_min_count=1 retains 23667 unique words (100% of original 23667, drops 0)
2020-01-20 13:29:14,931 : INFO : effective_min_count=1 leaves 157613 word corpus (100% of original 157613, drops 0)
2020-01-20 13:29:14,999 : INFO : deleting the raw counts dictionary of 23667 items
2020-01-20 13:29:15,000 : INFO : sample=0.001 downsamples 38 most-common words
2020-01-20 13:29:15,001 : INFO : downsampling leaves estimated 131462 word corpus (83.4% of prior 157613)
2020-01-20 13:29:15,041 : INFO : estimated required memory for 23667 words and 300 dimensions: 

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 300)         7100400   
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 1539      
Total params: 8,766,963
Trainable params: 1,666,563
Non-trainable params: 7,100,400
_________________________________________________________________
Epoch 1/10
7991/7991 [==============================] - 100s 13ms/step - loss: 0.4343 - acc: 0.8060
Epoch 2/10
7991/7991 [==============================] - 99s 12ms/step - loss: 0.3110 - acc: 0.8689
Epoch 3/10
7991/7991 [==============================] - 100s 12ms/step - loss: 0.2735 - acc: 0.8886
Epoch 4/10
7991/7991 [==============================] - 100s 12ms/s

2020-01-20 13:46:33,392 : INFO : Done


2664/2664 [==============================] - 54s 20ms/step
LABEL: positive
TEXT : gorselligi en ust seviyelere ulasmis ufak kizin muhtesem oyunculuguyla suslenmis muazzam bir sinema soleni
/n============================================
PREDICTION: {'label': 'negative', 'confidence': 0.5025225281715393, 'elapsed_time': 0.6448225975036621}


In [19]:
spam_clf.predict(4)

LABEL: negative
TEXT : bu filmin ikincisine hic gerek yokmus bence
/n============================================
PREDICTION: {'label': 'negative', 'confidence': 0.7360808253288269, 'elapsed_time': 0.6517021656036377}
